In [ ]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# Import common GIS tools
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import rioxarray as rio
import rasterio
from matplotlib.cm import RdYlGn,jet,RdBu

# Import Planetary Computer tools
import stackstac
import pystac_client
import planetary_computer
from odc.stac import stac_load
import geopandas as gpd
import pandas as pd

# Define the bounding box for the entire data region using (Latitude, Longitude)
# This is the region of New York City that contains our temperature dataset
lower_left = (40.75, -74.01)
upper_right = (40.88, -73.86)

# Calculate the bounds for doing an archive data search
# bounds = (min_lon, min_lat, max_lon, max_lat)
bounds = (lower_left[1], lower_left[0], upper_right[1], upper_right[0])

# Define the time window
time_window = "2021-06-01/2021-09-01"

stac = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

search = stac.search(
    bbox=bounds,
    datetime=time_window,
    collections=["sentinel-2-l2a"],
    query={"eo:cloud_cover": {"lt": 30}},
)

items = list(search.get_items())
print('This is the number of scenes that touch our region:',len(items))

# Define the pixel resolution for the final product
# Define the scale according to our selected crs, so we will use degrees
resolution = 10  # meters per pixel
scale = resolution / 111320.0 # degrees per pixel for crs=4326

data = stac_load(
    items,
    bands=["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B11", "B12"],
    crs="EPSG:2263", # Changed
    resolution=resolution, # Use meters directly
    chunks={"x": 2048, "y": 2048},
    dtype="uint16",
    patch_url=planetary_computer.sign,
    bbox=bounds
)

# Calculate NDVI
ndvi = (data.B08 - data.B04) / (data.B08 + data.B04)
ndvi = ndvi.compute() #calculate the result

# Find the time slice closest to 2021-07-24
target_date = pd.to_datetime("2021-07-24")
time_diffs = abs(data.time - target_date)
closest_time_index = time_diffs.argmin()

# Select the data for the closest time slice
ndvi_slice = ndvi.isel(time=closest_time_index)

# Save as Parquet
ndvi_df = ndvi_slice.to_dataframe(name='NDVI').reset_index()
gdf_ndvi = gpd.GeoDataFrame(ndvi_df, geometry=gpd.points_from_xy(ndvi_df.x, ndvi_df.y), crs="EPSG:2263")
gdf_ndvi.to_parquet('../output/sentinel2_ndvi.parquet')